In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import os

In [2]:
gender_separated = False # 是否區分男女
subject_separated = True # 是否區分科目

In [3]:
df = pd.read_csv("dataset/student_scores.csv")
df.head()

,id,first_name,last_name,email,gender,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,career_aspiration,math_score,history_score,physics_score,chemistry_score,biology_score,english_score,geography_score
0,1,Paul,Casey,paul.casey.1@gslingacademy.com,male,False,3,False,27,Lawyer,73,81,93,97,63,80,87
1,2,Danielle,Sandoval,danielle.sandoval.2@gslingacademy.com,female,False,2,False,47,Doctor,90,86,96,100,90,88,90
2,3,Tina,Andrews,tina.andrews.3@gslingacademy.com,female,False,9,True,13,Government Officer,81,97,95,96,65,77,94
3,4,Tara,Clark,tara.clark.4@gslingacademy.com,female,False,5,False,3,Artist,71,74,88,80,89,63,86
4,5,Anthony,Campos,anthony.campos.5@gslingacademy.com,male,False,5,False,10,Unknown,84,77,65,65,80,74,76


In [4]:
# Check for missing values
print("Missing Values:")
print(df.isnull().sum())

Missing Values:
id                            0
first_name                    0
last_name                     0
email                         0
gender                        0
part_time_job                 0
absence_days                  0
extracurricular_activities    0
weekly_self_study_hours       0
career_aspiration             0
math_score                    0
history_score                 0
physics_score                 0
chemistry_score               0
biology_score                 0
english_score                 0
geography_score               0
dtype: int64


In [5]:
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

def clean_data(df_sex, sex = "all"):

    # Drop irrelevant columns
    columns_to_drop = ["id", "first_name", "last_name", "email"]
    if gender_separated:
        columns_to_drop.append("gender")
    df_cleaned = df_sex.drop(columns=columns_to_drop, errors='ignore')

    # Encode categorical variables
    label_encoder = LabelEncoder()
    df_cleaned['part_time_job'] = label_encoder.fit_transform(df_cleaned['part_time_job'])
    df_cleaned['extracurricular_activities'] = label_encoder.fit_transform(df_cleaned['extracurricular_activities'])
    if not (gender_separated):
        df_cleaned['gender'] = label_encoder.fit_transform(df_cleaned['gender'])

    # One-hot encode career_aspiration
    df_cleaned = pd.get_dummies(df_cleaned, columns=["career_aspiration"], prefix="career")
    df_cleaned = df_cleaned.drop(columns=['career_Unknown'])
    career_columns = [col for col in df_cleaned.columns if col.startswith("career_")]
    df_cleaned[career_columns] = df_cleaned[career_columns].astype(int)

    df_cleaned = remove_outliers_iqr(df_cleaned, 'absence_days')
    df_cleaned = remove_outliers_iqr(df_cleaned, 'weekly_self_study_hours')

    if (subject_separated):
        subjects = ['math', 'history', 'physics', 'chemistry', 'biology', 'english', 'geography']
        cleaned_data = {subject: remove_outliers_iqr(df_cleaned, f'{subject}_score') for subject in subjects}
        
        base_path = "allgender_subject" if sex == "all" else f"gender_subject/{sex}"
        
        for subject, df in cleaned_data.items():
            score_col = f"{subject}_score"
            cols_to_keep = [col for col in df.columns if not col.endswith("_score") or col == score_col]
            df_filtered = df[cols_to_keep].copy()
            os.makedirs(base_path, exist_ok=True)
            df_filtered.to_csv(f"{base_path}/student_scores_clean_{subject}.csv", index=False)
            print(f"processed {sex} students {subject} data shape: {df.shape}")

    else:
        df_cleaned['average_score'] = df_cleaned[
            [f"{s}_score" for s in ['math', 'history', 'physics', 'chemistry', 'biology', 'english', 'geography']]
        ].mean(axis=1)
        df_cleaned = remove_outliers_iqr(df_cleaned, 'average_score')
        df_cleaned = df_cleaned.drop(columns=[f"{s}_score" for s in ['math', 'history', 'physics', 'chemistry', 'biology', 'english', 'geography']])
        return df_cleaned

In [6]:
if (gender_separated):
    for sex in ["male", "female"]:
        df_sex = df[df["gender"] == sex]
        print(f"{sex} students data shape:", df_sex.shape)
        result = clean_data(df_sex, sex)
        if not (subject_separated):
            os.makedirs(f"./gender_allsubject/{sex}", exist_ok=True)
            result.to_csv(f"./gender_allsubject/{sex}/student_scores_clean.csv", index=False)
            print(f"processed {sex} students data shape:", result.shape)
else:
    print("all students data shape:", df.shape)
    result = clean_data(df)
    if not subject_separated:
        os.makedirs("./allgender_allsubject", exist_ok=True)
        result.to_csv("./allgender_allsubject/student_scores_clean.csv", index=False)
        print("processed all students data shape:", result.shape)

all students data shape: (2000, 17)
processed all students math data shape: (1860, 28)
processed all students history data shape: (1912, 28)
processed all students physics data shape: (1912, 28)
processed all students chemistry data shape: (1912, 28)
processed all students biology data shape: (1893, 28)
processed all students english data shape: (1912, 28)
processed all students geography data shape: (1912, 28)
